In [51]:
import threading
import time
import random

# Thread objects

In [43]:
def f(*args):
    print("Thread\t\t: entering thread")
    time.sleep(1)
    print("Thread\t\t: executing with args =", args)
    print("Thread\t\t: exiting thread")


if __name__ == "__main__":
    # create a thread
    thread = threading.Thread(target=f, args=(1, ), daemon=False)
    # start a thread
    print("Main\t\t: starting thread")
    thread.start()  # call .start(), not .run()!
    # wait for thread to finish
    print("Main\t\t: waiting for thread")
    thread.join()
    print("Main\t\t: terminating")

Main		: starting thread
Thread		: entering thread
Main		: waiting for thread
Thread		: executing with args = (1,)
Thread		: exiting thread
Main		: terminating


# Lock objects

In [45]:
def f(lock, *args):
    print("Thread\t\t: entering thread")
    lock.acquire()
    time.sleep(1)
    print("Thread\t\t: executing with args =", args)
    lock.release()

    # use with to manage lock acquiring and releasing
    with lock:
        print("Thread\t\t: with block")

    print("Thread\t\t: exiting thread")

if __name__ == "__main__":
    # create a lock
    lock = threading.Lock()
    # acquire the lock
    lock.acquire()
    thread = threading.Thread(target=f, args=(lock, ), daemon=False)
    print("Main\t\t: starting thread")
    thread.start()
    time.sleep(1)
    # release the lock so that the thread can acquire it
    lock.release()
    print("Main\t\t: waiting for thread")
    thread.join()
    print("Main\t\t: terminating")

Main		: starting thread
Thread		: entering thread
Main		: waiting for thread
Thread		: executing with args = ()
Thread		: with block
Thread		: exiting thread
Main		: terminating


# RLock objects

In [46]:
def f(lock, *args):
    print("Thread\t\t: entering thread")
    with lock:
        time.sleep(1)
        print("Thread\t\t: executing with args =", args)
    print("Thread\t\t: exiting thread")

if __name__ == "__main__":
    # create a Rlock
    lock = threading.RLock()
    # acquire the lock twice
    lock.acquire()
    lock.acquire()  # does not block

    thread = threading.Thread(target=f, args=(lock, ), daemon=False)
    print("Main\t\t: starting thread")
    thread.start()

    # release the lock twice
    lock.release()  # the thead is still blocking
    time.sleep(1)
    lock.release()  # the lock is now fully released

    print("Main\t\t: waiting for thread")
    thread.join()
    print("Main\t\t: terminating")

Main		: starting thread
Thread		: entering thread
Main		: waiting for thread
Thread		: executing with args = ()
Thread		: exiting thread
Main		: terminating


The `RLock` can be acquired by the **owning thread** multiple times without blocking, but `Lock` blocks as long as it is acquired by a thread.

In [49]:
# An example trying to acquire a primitive lock twice in a thread

if __name__ == "__main__":
    # create a Lock
    lock = threading.Lock()
    # acquire the lock twice
    lock.acquire()
    try:
        lock.acquire(timeout=3)  # blocks forever if timeout is not set
    finally:
        print("Lock acquisition failed")

    lock.release()

Lock acquisition failed


# Semaphore objects

## How it works
- A semaphore manages an **atomic** counter which is decremented by each `acquire()` call and incremented by each `release()` call.
- The counter is always >= 0.
- When the counter is 0, `.acquire()` blocks until some other threads calls `.release()`.


## How to use
```
semaphore = theading.Semaphore(initial_counter)

semaphore.acquire()
semaphore.release()
```


## When to use
- Semaphores are frequently used to protect a resource that has a limited capacity.

# Condition objects

## How to use
1. `acquire()` and `release()`
2. `wait()` releases the lock and waits to be woken up
3. `notify()` wakes up a thread waiting on the condition, but does not release the lock
4. `notify_all()` wakes up all threads waiting for the condition.

## When to use
- The typical programming style using condition variables uses the lock to synchronize access to some shared state (see [details](https://docs.python.org/3/library/threading.html#condition-objects)).

## A producer-consumer example using threading.Condition

In [52]:
class SomeItem:
  # init method
  def __init__(self):
    # initialize empty list
    self.list = []

  # add to list method for producer
  def produce(self, item):
    print("Adding item to list...")
    self.list.append(item)

  # remove item from list method for consumer
  def consume(self):
    print("consuming item from list...")
    item = self.list[0]
    print("Item consumed: ", item)
    self.list.remove(item)

def producer(si, cond):
    r = random.randint(1,5)
    # creating random number of items
    for i in range(1, r):
      print("working on item creation, it will take: " + str(i) + " seconds")
      time.sleep(i)
      print("acquiring lock...")
      cond.acquire()
      try:
        si.produce(i)
        cond.notify()  # wakes up a waiting thread, but yet to release the lock
      finally:
        cond.release()  # explicitly releases the lock

def consumer(si, cond):
    cond.acquire()
    while True:
      try:
        si.consume()
      except:
        print("No item to consume...")
        # wait with a maximum timeout of 10 sec
        val = cond.wait(10)   # releases the lock
        if val:
          print("notification received about item production...")
          continue
        else:
          print("waiting timeout...")
          break

    cond.release()

if __name__=='__main__':
  # condition object
  cond = threading.Condition()
  # SomeItem object
  si = SomeItem()
  # producer thread
  p = threading.Thread(target=producer, args=(si,cond,))
  p.start()
  # consumer thread
  c = threading.Thread(target=consumer, args=(si,cond,))
  c.start()

  #print('Waiting for producer and consumer threads...')
  p.join()
  c.join()
  print("Done")

working on item creation, it will take: 1 seconds
consuming item from list...
No item to consume...
acquiring lock...
Adding item to list...
working on item creation, it will take: 2 seconds
notification received about item production...
consuming item from list...
Item consumed:  1
consuming item from list...
No item to consume...
acquiring lock...
Adding item to list...
notification received about item production...
consuming item from list...
Item consumed:  2
consuming item from list...
No item to consume...
waiting timeout...
Done
